In [31]:
import numpy as np

In [34]:
class Hull_painting_robot:
    def __init__(self, intcode, initial_i, initial_j, hull):
        self.intcode = intcode
        self.intcode.extend([0 for i in range(100*len(intcode))])
        self.relative_base = 0
        self.program_counter = 0
        self.i = initial_i
        self.j = initial_j
        self.direction = "up"
        self.hull = hull
        self.paint_move = 0
        self.painted_cells = []
    def run_program(self):
        #Execute the program
        while True:
            opcode=int(str(self.intcode[self.program_counter])[-2:])
            #Extremely unefficient switch:
            #3 parameter instructions
            if opcode==1 or opcode==2 or opcode==7 or opcode==8:
                #print("Opcode: " + str(intcode[program_counter]))
                param_1_mode=int(str(self.intcode[self.program_counter])[-3]) if\
                len(str(self.intcode[self.program_counter]))>=3 else 0
                param_2_mode=int(str(self.intcode[self.program_counter])[-4]) if\
                len(str(self.intcode[self.program_counter]))>=4 else 0
                param_3_mode=int(str(self.intcode[self.program_counter])[-5]) if\
                len(str(self.intcode[self.program_counter]))>=5 else 0

                param_1=self.get_value(param_1_mode, self.intcode[self.program_counter+1])
                param_2=self.get_value(param_2_mode, self.intcode[self.program_counter+2])

                #The last parameter is for writing, so we don't need the value of the
                #position it points to but the index of that position, which will be
                #the parameter itself in mode 0 and the parameter plus the base in mode 2
                if param_3_mode==0:
                    param_3=intcode[self.program_counter+3]
                elif param_3_mode==2:
                    param_3=intcode[self.program_counter+3]+self.relative_base
                #ADD
                if opcode==1:
                    self.intcode[param_3]=param_1+param_2
                #PRODUCT
                elif opcode==2:
                    self.intcode[param_3]=param_1*param_2
                #LESS THAN
                elif opcode==7:
                    self.intcode[param_3]=1 if param_1<param_2 else 0
                #EQUALS
                elif opcode==8:
                    self.intcode[param_3]=1 if param_1==param_2 else 0
                self.program_counter+=4
            #2 parameter instructions
            elif opcode==5 or opcode==6:
                param_1_mode=int(str(self.intcode[self.program_counter])[-3]) if\
                len(str(self.intcode[self.program_counter]))>=3 else 0
                param_2_mode=int(str(self.intcode[self.program_counter])[-4]) if\
                len(str(self.intcode[self.program_counter]))>=4 else 0
                param_1=self.get_value(param_1_mode, self.intcode[self.program_counter+1])
                param_2=self.get_value(param_2_mode, self.intcode[self.program_counter+2])
                #JUMP IF TRUE
                if opcode==5:
                    if param_1!=0:
                        self.program_counter=param_2
                    else:
                        self.program_counter+=3
                #JUMP IF FALSE
                elif opcode==6:
                    if param_1==0:
                        self.program_counter=param_2
                    else:
                        self.program_counter+=3

            #1 parameter instructions
            elif opcode==3 or opcode==4 or opcode==9:
                #READ
                if opcode==3:
                    param_1_mode=int(str(self.intcode[self.program_counter])[-3]) if\
                    len(str(self.intcode[self.program_counter]))>=3 else 0
                    if param_1_mode==0:
                        self.intcode[self.intcode[self.program_counter+1]] = hull[self.i][self.j]
                    elif param_1_mode==2:
                        self.intcode[self.intcode[self.program_counter+1]+self.relative_base] =\
                            hull[self.i][self.j]
                #WRITE
                elif opcode==4:
                    param_1_mode=int(str(self.intcode[self.program_counter])[-3]) if\
                    len(str(self.intcode[self.program_counter]))>=3 else 0
                    param_1=self.get_value(param_1_mode, self.intcode[self.program_counter+1])
                    #print(param_1)
                    self.paint_or_move(param_1)
                #INCREASE RELATIVE BASE VALUE
                elif opcode==9:
                    param_1_mode=int(str(self.intcode[self.program_counter])[-3]) if\
                    len(str(self.intcode[self.program_counter]))>=3 else 0
                    param_1=self.get_value(param_1_mode, self.intcode[self.program_counter+1])
                    self.relative_base+=param_1
                self.program_counter+=2
            elif opcode==99:
                print("Program complete")
                break
            else:
                print("An error has occurred")
                break
    def get_value(self, parameter_mode, parameter):
        '''
        Returns the value of the parameter based on it's mode. For mode
        0 the value returned is the value stored in the memory (intcode)
        position indicated by the parameter. For mode 1, the value of
        the parameter is returned
        '''
        if parameter_mode==0:
            return self.intcode[parameter]
        elif parameter_mode==1:
            return parameter
        elif parameter_mode==2:
            return self.intcode[parameter+self.relative_base]
    
    def paint_or_move(self, param):
        if self.paint_move==0:
            hull[self.i][self.j]=param
            if (self.i, self.j) not in self.painted_cells:
                self.painted_cells.append((self.i, self.j))
            self.paint_move=1
        else:
            if param==0:
                if self.direction=="up":
                    self.direction="left"
                    self.j-=1
                elif self.direction=="right":
                    self.direction="up"
                    self.i-=1
                elif self.direction=="down":
                    self.direction="right"
                    self.j+=1
                elif self.direction=="left":
                    self.direction="down"
                    self.i+=1
            else:
                if self.direction=="up":
                    self.direction="right"
                    self.j+=1
                elif self.direction=="right":
                    self.direction="down"
                    self.i+=1
                elif self.direction=="down":
                    self.direction="left"
                    self.j-=1
                elif self.direction=="left":
                    self.direction="up"
                    self.i-=1
            self.paint_move=0
            

In [35]:
f=open("input", "r")
intcode = [int(i) for i in f.read().split(",")]
f.close()
hull=np.zeros((100,100))
robot=Hull_painting_robot(intcode,50,50,hull)
robot.run_program()

Program complete


In [37]:
len(robot.painted_cells)

2226